In [21]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np

In [22]:
fch = pd.read_csv("data/FINAL_FEATURES_TRAINTEST.tsv",sep='\t')
trg = pd.read_csv("data/FINAL_TARGETS_DATES_TRAINTEST.tsv",sep="\t")

In [ ]:
new_fch_trg = pd.merge(trg, fch, on="CLIENT_ID",how="left")

In [ ]:
new_fch_trg.drop(["CLIENT_ID"],axis=1,inplace=True)
new_fch_trg.drop(["RETRO_DT"],axis=1,inplace=True)

In [ ]:
new_fch_trg.fillna(0)
new_test = new_fch_trg[(new_fch_trg["TARGET"]=='test')]
new_fch_trg.drop(new_fch_trg.index[(new_fch_trg["TARGET"]=='test')],axis=0,inplace=True)
new_fch = new_fch_trg.drop(["TARGET"],axis=1)
new_trg = new_fch_trg["TARGET"]

In [27]:
cat = CatBoostClassifier()
#cat.fit(X=new_fch,y=new_trg,verbose=False)

In [ ]:
new_test_fch = new_test.drop(["TARGET"],axis=1)
new_test_fch

In [ ]:
pred = cat.predict_proba(new_fch,verbose=False)
pred[::1]

In [ ]:
roc_auc_score(new_trg,pred[:, 1])

In [24]:
join_fch_trg = pd.merge(trg, fch, on="CLIENT_ID",how="left")
join_fch_trg.drop(["CLIENT_ID"],axis=1,inplace=True)
join_fch_trg.drop(["RETRO_DT"],axis=1,inplace=True)
join_fch_trg.drop(join_fch_trg.index[(join_fch_trg["TARGET"]=='test')],axis=0,inplace=True)
witout_null = join_fch_trg.dropna(axis=0)
with_null = join_fch_trg.fillna(0)
#join_fch_trg = join_fch_trg.reset_index()
mask = np.random.choice(join_fch_trg.shape[0], size=join_fch_trg.shape[0], replace=False)
ratio = int(0.8 * len(mask))
train_dataset, val_dataset = join_fch_trg.iloc[list(mask[:ratio])], join_fch_trg.iloc[list(mask[ratio:])]

### С нулями

In [34]:
mask = np.random.choice(with_null.shape[0], size=with_null.shape[0], replace=False)
ratio = int(0.8 * len(mask))
train_null_dataset, val_null_dataset = with_null.iloc[list(mask[:ratio])], with_null.iloc[list(mask[ratio:])]
train_null_fch = train_null_dataset.drop(["TARGET"],axis=1)
train_null_trg = train_null_dataset["TARGET"]
val_null_fch = val_null_dataset.drop(["TARGET"],axis=1)
val_null_trg = val_null_dataset["TARGET"]
mask = np.random.choice(witout_null.shape[0], size=witout_null.shape[0], replace=False)
ratio = int(0.8 * len(mask))
train_wnull_dataset, val_wnull_dataset = witout_null.iloc[list(mask[:ratio])], witout_null.iloc[list(mask[ratio:])]
train_wnull_fch = train_wnull_dataset.drop(["TARGET"],axis=1)
train_wnull_trg = train_wnull_dataset["TARGET"]
val_wnull_fch = val_wnull_dataset.drop(["TARGET"],axis=1)
val_wnull_trg = val_wnull_dataset["TARGET"]

cat.fit(X=train_null_fch,y=train_null_trg,verbose=False)
cat.fit(X=train_wnull_fch,y=train_wnull_trg,verbose=False)


In [37]:
new_prd = cat.predict_proba(val_null_fch,verbose=False)
roc_auc_score(val_null_trg,new_prd[:,1])

0.6743723386059519

In [ ]:
train_fch = train_dataset.drop(["TARGET"],axis=1)
train_trg = train_dataset["TARGET"]
cat.fit(X=train_fch,y=train_trg,verbose=False)

In [31]:
val_fch = val_dataset.drop(["TARGET"],axis=1)
val_trg = val_dataset["TARGET"]
new_prd = cat.predict_proba(val_fch,verbose=False)

In [32]:
roc_auc_score(val_trg,new_prd[:,1])

0.6949126664205015

In [ ]:
submission = pd.read_csv("sample_submission.csv")

In [ ]:
features = pd.merge(submission, fch, on="CLIENT_ID", how="left")
X = features.drop("CLIENT_ID", axis=1)
submission["TARGET"] = cat.predict_proba(X)[:, 1]
submission

In [ ]:
submission.to_csv("submission.csv", index=False)

In [ ]:
pd.merge(submission, cat.predict_proba(X)[:, 1], on="CLIENT_ID")